深入了解用户画像及行为偏好，挖掘出影响用户流失的关键因素

作者：云朵君  
公众号：数据STUDIO  
代码仓促整理，若有问题，请及时联系云朵君  

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
plt.rcParams['font.family'] = ['Arial Unicode MS']#显示中文,解决图中无法显示中文的问题
plt.rcParams['axes.unicode_minus']=False

# 数据清洗

## 导入数据

In [3]:
Xtrain_new = pd.read_csv('Xtrain_new.csv',index_col=0)
Xtest_new = pd.read_csv('Xtest_new.csv',index_col=0)
Ytrain = pd.read_csv('Ytrain.csv',index_col=0,header=None)
Ytest = pd.read_csv('Ytest.csv',index_col=0,header=None)

FileNotFoundError: [Errno 2] No such file or directory: 'Xtrain_new.csv'

In [ ]:
Ytest = Ytest.reset_index(drop=True)

In [ ]:
Ytest = pd.Series(Ytest.values.reshape(1,-1)[0],name='label')

In [ ]:
Ytrain = pd.Series(Ytrain.values.reshape(1,-1)[0])

In [ ]:
date_train = pd.read_csv('date_train.csv',index_col=0)
date_test = pd.read_csv('date_test.csv',index_col=0)

In [ ]:
Xtrain_new.shape

In [ ]:
date_train.shape

## 分箱

In [ ]:
features =  {'lasthtlordergap':'一年内距离上次下单时长'
             ,'cityorders':'昨日提交当前城市同入住日期的app订单数','lastpvgap':'一年内距上次访问时长'
            ,'cr':'用户转化率','sid':'会话id，sid=1可认为是新访','visitnum_oneyear':'年访问次数'
            ,'h':'访问时间点'}

In [ ]:
pd.Series(features)

In [ ]:
Ytrain.value_counts()

- 连续型变量分箱：lasthtlordergap/lastpvgap  --时长
- 分类型变量分箱：sid / h

In [ ]:
%run smob.py

In [ ]:
ytrain = 1-Ytrain
ytest = 1-Ytest

In [ ]:
train_data = pd.concat([Xtrain_new,ytrain],axis=1)
train_data.rename(columns = {0:'label'},inplace=True)
y = 'label'

In [ ]:
train_data.head()

In [ ]:
test_data = pd.concat([Xtest_new,ytest],axis=1)
test_data.rename(columns = {0:'label'},inplace=True)
test_data.head()

In [ ]:
import seaborn as sns

In [ ]:
l_ = ['一年内距离上次下单时长'
,'昨日提交当前城市同入住日期的app订单数'
,'一年内距上次访问时长'
,'用户转化率'
,'会话id,sid=1可认为是新访'
 ,'年访问次数'
 ,'访问时间点']

In [ ]:
ax = plt.figure(figsize=(8,15))
for i in range(len(train_data.columns[:-1])):
    ax.add_subplot(8,1,i+1)
    sns.kdeplot(train_data.iloc[:,i],shade=True)
    plt.title(l_[i])
plt.tight_layout();

In [ ]:
ax = plt.figure(figsize=(8,15))
for i in range(len(train_data.columns[:-1])):
    ax.add_subplot(8,1,i+1)
    train_data.iloc[:,i].plot(kind='hist',bins=20)
plt.tight_layout();

In [ ]:
len(train_data.columns[:-1])

### WOE分箱

In [ ]:
iv_all = pd.Series()

In [ ]:
ordergap = smbin(train_data,y,'lasthtlordergap',max_bin = 8)

In [ ]:
ordergap.IVtable

In [ ]:
iv_all = iv_all.append(pd.Series({'lasthtlordergap':ordergap.IV}))
iv_all

**lasthtlordergap 8 箱**

In [ ]:
cityorders = smbin(train_data,y,'cityorders',max_bin=8)

In [ ]:
cityorders.IVtable

In [ ]:
iv_all = iv_all.append(pd.Series({'cityorders':cityorders.IV}))
iv_all

In [ ]:
lastpvgap = smbin(train_data,y,'lastpvgap',max_bin=10)

In [ ]:
lastpvgap.IVtable

In [ ]:
iv_all = iv_all.append(pd.Series({'lastpvgap':lastpvgap.IV}))
iv_all

In [ ]:
cr = smbin(train_data,y,'cr',max_bin =8)

In [ ]:
cr.IVtable

In [ ]:
iv_all = iv_all.append(pd.Series({'cr':cr.IV}))
iv_all

In [ ]:
sid  =smbin(train_data,y,'sid',max_bin=10)

In [ ]:
sid.IVtable

In [ ]:
iv_all = iv_all.append(pd.Series({'sid':sid.IV}))
iv_all

In [ ]:
visitnum = smbin(train_data,y,'visitnum_oneyear',max_bin=7)

In [ ]:
visitnum.IVtable

In [ ]:
iv_all = iv_all.append(pd.Series({'visitnum':visitnum.IV}))
iv_all

In [ ]:
h = smbin(train_data,y,'h',max_bin=8)

In [ ]:
h.IVtable

In [ ]:
iv_all = iv_all.append(pd.Series({'h':h.IV}))
iv_all

**除去IV < 0.02的变量**

In [ ]:
bin_list = [ordergap,cityorders,cr,visitnum,h]

In [ ]:
iv_all.sort_values().plot(kind='barh')

In [ ]:
out = iv_all.sort_values(ascending=False).index[:-3:-1]
out

In [ ]:
iv_list = iv_all.drop(index=out)

## 模型构建

In [ ]:
features_02 =  {'lasthtlordergap':'一年内距离上次下单时长'
             ,'cityorders':'昨日提交当前城市同入住日期的app订单数'
            ,'cr':'用户转化率','visitnum_oneyear':'年访问次数','h':'访问时间点'}
#'lastpvgap':'一年内距上次访问时长','sid':'会话id，sid=1可认为是新访',

In [ ]:
train_data.head()

In [ ]:
train_data_ = train_data.drop(columns=out,axis=1)

In [ ]:
train_data_.head()

In [ ]:
data_woe = smgen(train_data_,bin_list)
data_woe.head()

In [ ]:
X_train = data_woe.iloc[:,-len(bin_list):]
X_train.head()

In [ ]:
Y_train = data_woe[y]
Y_train

In [ ]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression().fit(X_train,Y_train)

In [ ]:
LR = LogisticRegression().fit(X_train,Y_train)

In [ ]:
LR.score(X_train,Y_train)

In [ ]:
LR.coef_

In [ ]:
aa = pd.DataFrame({'LR.score(X_train,Y_train)':LR.score(X_train,Y_train),'LR.score(X_test,Y_test)':LR.score(X_test,Y_test)},index=['score'])

bb = pd.DataFrame({'intercept_':LR.intercept_[0]},index=['coef_'])
bb

In [ ]:
aa

In [ ]:
pd.concat([pd.DataFrame(dict(zip(X_train.columns,LR.coef_[0])),index=['coef_']),bb],axis=1)

In [ ]:
pd.DataFrame(dict(zip(X_train.columns,LR.coef_[0])),index=['coef_'])

## 模型评估

In [ ]:
test_data = test_data.drop(columns=out,axis=1)

In [ ]:
test_woe = smgennew(test_data,bin_list)
test_woe.head()

In [ ]:
X_test = test_woe.iloc[:,-len(bin_list):]

In [ ]:
X_test

In [ ]:
Y_test = test_woe.loc[:,y]
Y_test

In [ ]:
LR.score(X_test,Y_test)

In [ ]:
from sklearn.metrics import auc, roc_curve

In [ ]:
y_score

In [ ]:
y_pred

In [ ]:
# ROC
y_score = LR.predict_proba(X_test)  # 随机森林
fpr, tpr, thresholds = roc_curve(Y_test, y_score[:, 1])
roc_auc = auc(fpr, tpr)
def drawRoc(roc_auc,fpr,tpr):
    plt.subplots(figsize=(7, 5.5))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc="lower right")
    plt.show()
drawRoc(roc_auc, fpr, tpr)

In [ ]:
from sklearn.metrics import accuracy_score, auc, confusion_matrix, f1_score, \
    precision_score, recall_score, roc_curve
import prettytable

In [ ]:
y_pred = LR.predict(X_test)

In [ ]:
y_score = LR.predict_proba(X_test)  # 随机森林
fpr, tpr, thresholds = roc_curve(Y_test, y_score[:, 1])
roc_auc = auc(fpr, tpr)
y_pred = LR.predict(X_test)
scores = [round(i(Y_test, y_pred),3 )for  i in (accuracy_score,precision_score,\
                                     recall_score,f1_score)]
scores.insert(0,roc_auc)
core_metrics = prettytable.PrettyTable()  # 创建表格实例
core_metrics.field_names = ['auc', 'accuracy', 'precision', 'recall', 'f1']  # 定义表格列名
core_metrics.add_row(scores)  # 增加数据
print('core metrics\n',core_metrics)

# 特征创造1

In [ ]:
M_train = pd.read_csv('M_train.csv',index_col=0,header=None)

In [ ]:
M_test = pd.read_csv('M_test.csv',index_col=0,header=None)

In [ ]:
M_train

In [ ]:
train_data.head()

In [ ]:
train_data_m = pd.concat([train_data,M_train],axis=1)

In [ ]:
train_data_m.rename(columns = {1:'M'},inplace=True)
train_data_m.head()

In [ ]:
M = smbin(train_data_m,y,'M',max_bin=8)

In [ ]:
M.IVtable

In [ ]:
iv_all.append(pd.Series({"M":M.IV}))

In [ ]:
bin_list_m = [ordergap,cityorders,cr,visitnum,h,M]

In [ ]:
train_data_m_ = train_data_m.drop(columns=out,axis=1)

In [ ]:
data_woe_m = smgen(train_data_m_,bin_list_m)
data_woe_m.head()

In [ ]:
X_train_m = data_woe_m.iloc[:,-len(bin_list_m):]
X_train_m.head()

In [ ]:
LR_m = LogisticRegression().fit(X_train_m,Y_train)

In [ ]:
LR_m.score(X_train_m,Y_train)

In [ ]:
test_data_m = pd.concat([test_data,M_test],axis=1)
test_data_m.rename(columns = {1:'M'},inplace=True)
test_data_m.head()

In [ ]:
test_woe_m = smgennew(test_data_m,bin_list_m)
test_woe_m.head()

In [ ]:
X_test_m = test_woe_m.iloc[:,-len(bin_list_m):]
X_test_m.head()

In [ ]:
Y_test.shape

In [ ]:
X_test_m.shape

In [ ]:
LR_m.score(X_test_m,Y_test)

In [ ]:
# ROC
y_score_m = LR_m.predict_proba(X_test_m)  # 随机森林
fpr, tpr, thresholds = roc_curve(Y_test, y_score_m[:, 1])
roc_auc = auc(fpr, tpr)
def drawRoc(roc_auc,fpr,tpr):
    plt.subplots(figsize=(7, 5.5))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc="lower right")
    plt.show()
drawRoc(roc_auc, fpr, tpr)

In [ ]:
bbb = pd.DataFrame({'intercept_':LR_m.intercept_[0]},index=['coef_'])
bbb

pd.concat([pd.DataFrame(dict(zip(X_train_m.columns,LR_m.coef_[0])),index=['coef_']),bbb],axis=1)

bbb

aaa = pd.DataFrame({'LR.score(X_train,Y_train)':LR_m.score(X_train_m,Y_train),'LR.score(X_test,Y_test)':LR_m.score(X_test_m,Y_test)},index=['score'])

In [ ]:
aaa

In [ ]:
y_score = LR_m.predict_proba(X_test_m)  # 随机森林
fpr, tpr, thresholds = roc_curve(Y_test, y_score[:, 1])
roc_auc = auc(fpr, tpr)
y_pred = LR_m.predict(X_test_m)
scores = [round(i(Y_test, y_pred),3 )for  i in (accuracy_score,precision_score,\
                                     recall_score,f1_score)]
scores.insert(0,roc_auc)
core_metrics = prettytable.PrettyTable()  # 创建表格实例
core_metrics.field_names = ['auc', 'accuracy', 'precision', 'recall', 'f1']  # 定义表格列名
core_metrics.add_row(scores)  # 增加数据
print('core metrics\n',core_metrics)

# 特征创造2

In [ ]:
import datetime

In [ ]:
date_train

In [ ]:
date_train.d.apply(str_date)
date_train.arrival.apply(str_date)

In [ ]:
date_test['delta'] = date_test.arrival.map(str_date) - date_test.d.map(str_date)

In [ ]:
date_train['delta'] = date_train.arrival.map(str_date) - date_train.d.map(str_date)

date_train['delta'] = date_train['delta'].dt.days

In [ ]:
date_train = date_train.reset_index(drop=True)

In [ ]:
date_test = date_test.reset_index(drop=True)

In [ ]:
def str_date(x):
    out = datetime.datetime.strptime(x,"%Y-%m-%d").date()
    return out

In [ ]:
def date_str(x):
    out = int(x)
    return out

In [ ]:
str_date('2010-10-9')

In [ ]:
train_data_d = pd.concat([train_data_m,date_train.delta],axis=1)

In [ ]:
test_data_d = pd.concat([test_data_m,date_test.delta],axis=1)

In [ ]:
train_data_d['delta'] = train_data_d['delta'].dt.days

In [ ]:
bin_list_d = [ordergap,cityorders,cr,visitnum,h,M,delta]

In [ ]:
delta = smbin(train_data_d,y,'delta',max_bin=7)

In [ ]:
delta.IVtable

In [ ]:
iv_all.append(pd.Series({"delta":delta.IV}))

In [ ]:
plt.figure(figsize=(8,6))
iv_all.sort_values().plot.barh(label='IV值')
plt.yticks(fontsize=15)
plt.xticks(fontsize=13)
plt.legend();

In [ ]:
train_data_d_ = train_data_d.drop(columns=out,axis=1)

In [ ]:
data_woe_d = smgen(train_data_d_,bin_list_d)
data_woe_d.head()

In [ ]:
X_train_d = data_woe_d.iloc[:,-len(bin_list_d):]
X_train_d.head()

In [ ]:
LR_d = LogisticRegression().fit(X_train_d,Y_train)

In [ ]:
LR_d.score(X_train_d,Y_train)

In [ ]:
date_test['delta'] = date_test['delta'].dt.days

In [ ]:
test_data_d = pd.concat([test_data_m,date_test['delta']],axis=1)

In [ ]:
test_data_d.head()

In [ ]:
test_woe_d = smgennew(test_data_d,bin_list_d)
test_woe_d.head()

In [ ]:
X_test_d = test_woe_d.iloc[:,-len(bin_list_d):]
X_test_d.head()

In [ ]:
LR_d.score(X_test_d,Y_test)

In [ ]:
# ROC
y_score_d = LR_d.predict_proba(X_test_d)  # 随机森林
fpr, tpr, thresholds = roc_curve(Y_test, y_score_d[:, 1])
roc_auc = auc(fpr, tpr)
def drawRoc(roc_auc,fpr,tpr):
    plt.subplots(figsize=(7, 5.5))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc="lower right")
    plt.show()
drawRoc(roc_auc, fpr, tpr)

In [ ]:
bbbb = pd.DataFrame({'intercept_':LR_d.intercept_[0]},index=['coef_'])
bbbb

In [ ]:
pd.concat([pd.DataFrame(dict(zip(X_train_d.columns,LR_d.coef_[0])),index=['coef_']),bbbb],axis=1)

In [ ]:
aaaa = pd.DataFrame({'LR.score(X_train,Y_train)':LR_d.score(X_train_d,Y_train),'LR.score(X_test,Y_test)':LR_d.score(X_test_d,Y_test)},index=['score'])

In [ ]:
aaaa

In [ ]:
y_score_d = LR_d.predict_proba(X_test_d)  # 随机森林
fpr, tpr, thresholds = roc_curve(Y_test, y_score_d[:, 1])
roc_auc = auc(fpr, tpr)
y_pred = LR_d.predict(X_test_d)
scores = [round(i(Y_test, y_pred),3 )for  i in (accuracy_score,precision_score,\
                                     recall_score,f1_score)]
scores.insert(0,roc_auc)
core_metrics = prettytable.PrettyTable()  # 创建表格实例
core_metrics.field_names = ['auc', 'accuracy', 'precision', 'recall', 'f1']  # 定义表格列名
core_metrics.add_row(scores)  # 增加数据
print('core metrics\n',core_metrics)